In [1]:
from firecloud import fiss
import firecloud.api as fapi
import os
import io
import pandas as pd
import argparse
import sys 
import json


In [2]:
sys.path.append("/Users/fkhajoue/GitHub/warp/pipelines/skylab/optimus/scripts/")

In [3]:
import workspace_controller as terra

In [4]:
# Get the Google billing project name and workspace name
billing_project = "hca-dev-staging-billing"
workspace_with_workflow = "DCP2_Optimus_template_FK"
workspace_with_data = "DCP2_Optimus_template_FK_data"
#Set the new workspace name to project label from the spreadsheet
new_workspace_name = "FK-test-optimus-automation"

# Clone a workspace with the workflows

In [5]:
response = terra.create_newworkspace(billing_project, workspace_with_workflow, new_workspace_name)

if "attributes" in response.json():
    print("success!")

# Get the participant table from the data workspace and save it to a file

In [11]:
#set these paths to a local directory to save the data table into and modify the output
input_participant_table_name = "/Users/fkhajoue/Downloads/participant.tsv"
output_prefix = "/Users/fkhajoue/GitHub/Scratch/data/project_label"

In [6]:
terra.get_data_from_workspace(billing_project, workspace_with_data, "participant",
                              input_participant_table_name)

updated successfully


<Response [200]>

# Modify the participant table for running Optimus

In [12]:
terra.parse_terra.create_output_files(input_participant_table_name, output_prefix)

# Upload the modified data into your new workspace

In [13]:
terra.upload_tables(output_prefix + ".tsv", billing_project, new_workspace_name)
terra.upload_tables(output_prefix + "_entity.tsv", billing_project, new_workspace_name)
terra.upload_tables(output_prefix + "_membership.tsv", billing_project, new_workspace_name)

Updating participant_lane entitys 1-20, batch 1/1
Updating participant_lane_set entitys 1-20, batch 1/1
Updating participant_lane_set memberships 1-20, batch 1/1


# Modify json template to have appropriate info about the pipeline set up for the data

In [5]:
import json_templates

In [36]:
#Change the chemistry to v3 if necessary
json_templates.optimus_inputs['Optimus.chemistry'] = '\"tenX_v3\"'
json_templates.optimus_inputs['Optimus.whitelist'] = 'workspace.whitelist_v3'

In [37]:
#Change counting mode to sn_rna if necssary
json_templates.optimus_inputs['Optimus.counting_mode'] = '\"sn_rna\"'

In [38]:
#Change reference to mouse
json_templates.optimus_inputs['Optimus.annotations_gtf'] = 'workspace.mouse_annotations_gtf'
json_templates.optimus_inputs['Optimus.ref_genome_fasta'] = 'workspace.mouse_ref_genome_fasta'
json_templates.optimus_inputs['Optimus.tar_star_reference'] = 'workspace.mouse_tar_star_reference'

In [6]:
json_templates.optimus_inputs

{'Optimus.annotations_gtf': 'workspace.human_annotations_gtf',
 'Optimus.chemistry': '"tenX_v2"',
 'Optimus.counting_mode': '"sc_rna"',
 'Optimus.emptydrops_lower': '1',
 'Optimus.i1_fastq': 'this.participant_lanes.i1_fastq',
 'Optimus.input_id': 'this.participant_lane_set_id',
 'Optimus.input_id_metadata_field': '"sequencing_process.provenance.document_id"',
 'Optimus.input_name': 'this.input_name',
 'Optimus.input_name_metadata_field': '"sequencing_input.biomaterial_core.biomaterial_id"',
 'Optimus.r1_fastq': 'this.participant_lanes.r1_fastq',
 'Optimus.r2_fastq': 'this.participant_lanes.r2_fastq',
 'Optimus.ref_genome_fasta': 'workspace.human_ref_genome_fasta',
 'Optimus.tar_star_reference': 'workspace.human_tar_star_reference',
 'Optimus.whitelist': 'workspace.whitelist_v2'}

In [17]:
workflow_name = "Optimus_4.1.7/Optimus_4.1.7/1"

In [18]:
terra.upload_workflow_json(billing_project,new_workspace_name,workflow_name,json_templates)

TypeError: can only concatenate str (not "bytes") to str

In [19]:
work_space_config = fapi.get_workspace_config(billing_project, new_workspace_name, new_workspace_name, "Optimus_4.1.7")

In [20]:
work_space_json = work_space_config.json()

In [21]:
work_space_json

{'statusCode': 404,
 'source': 'rawls',
 'timestamp': 1605209069145,
 'causes': [],
 'stackTrace': [],
 'message': 'FK-test-optimus-automation/Optimus_4.1.7 does not exist in Workspace(hca-dev-staging-billing,FK-test-optimus-automation,92a4c21d-bc95-44da-868b-e0bac952ed2a,fc-92a4c21d-bc95-44da-868b-e0bac952ed2a,Some(92a4c21d-bc95-44da-868b-e0bac952ed2a),2020-11-12T17:36:32.282Z,2020-11-12T19:22:10.410Z,fkhajoue@broadinstitute.org,Map(AttributeName(default,referenceData_hg38_one_thousand_genomes_resource_vcf_index) -> AttributeString(gs://gcp-public-data--broad-references/hg38/v0/1000G_phase1.snps.high_confidence.hg38.vcf.gz.tbi), AttributeName(default,referenceData_hg38_ref_pac) -> AttributeString(gs://gcp-public-data--broad-references/hg38/v0/Homo_sapiens_assembly38.fasta.64.pac), AttributeName(default,referenceData_hg38_hapmap_resource_vcf) -> AttributeString(gs://gcp-public-data--broad-references/hg38/v0/hapmap_3.3.hg38.vcf.gz), AttributeName(default,referenceData_hg38_ref_amb) -> A